In [1]:

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import train_test_split
import numpy as np
from utils.Preprocess import Preprocess
import os
import pandas as pd



In [2]:
# 학습 파일 불러오기
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

# 전처리 모듈
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

# ① 학습용 말뭉치 데이터를 불러옴
import pickle
with open('tsv_ner.pkl','rb') as f:
    tsv_ner = pickle.load(f)


In [3]:
p.pos('상의')

[('상의', 'NNG')]

In [4]:
tsv_ner

[[('1', '신발', 'NNG', 'B_CATEGORY'),
  ('2', '주문', 'NNP', 'O'),
  ('3', '하', 'VV', 'O'),
  ('4', '고', 'EC', 'O'),
  ('5', '싶', 'VX', 'O'),
  ('6', '어요', 'EC', 'O')],
 [('1', '주얼리', 'NNP', 'B_CATEGORY'),
  ('2', '먹', 'VV', 'O'),
  ('3', '고', 'EC', 'O'),
  ('4', '싶', 'VX', 'O'),
  ('5', '어요', 'EC', 'O')],
 [('1', '원피스', 'NNG', 'B_CATEGORY'),
  ('2', '하', 'VV', 'O'),
  ('3', '고', 'EC', 'O'),
  ('4', '싶', 'VX', 'O'),
  ('5', '어요', 'EC', 'O')],
 [('1', '주얼리', 'NNP', 'B_CATEGORY'),
  ('2', '원', 'NNB', 'O'),
  ('3', '하', 'XSA', 'O'),
  ('4', '어요', 'EC', 'O')],
 [('1', '바지', 'NNG', 'B_CATEGORY'),
  ('2', '어떻', 'VA', 'O'),
  ('3', '게', 'EC', 'O'),
  ('4', '하', 'VV', 'O'),
  ('5', '아야', 'EC', 'O'),
  ('6', '되', 'VV', 'O'),
  ('7', '요', 'EF', 'O'),
  ('8', '?', 'SF', 'O')],
 [('1', '바지', 'NNG', 'B_CATEGORY'),
  ('2', '어떻', 'VA', 'O'),
  ('3', '게', 'EC', 'O'),
  ('4', '하', 'VV', 'O'),
  ('5', '아야', 'EC', 'O'),
  ('6', '되', 'VV', 'O'),
  ('7', '어', 'EC', 'O')],
 [('1', '뷰티', 'NNP', 'B_CATEGORY'),
  

In [5]:
# ② 위에서 불러온 말뭉치 데이터에서 단어(w[1])와 BIO 태그(w[3])만 불러와 학습용 데이터셋 생성

sentences, tags = [], []
for t in tsv_ner:
    tagged_sentence = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentence.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])
    
    sentences.append(sentence)
    tags.append(bio_tag)



In [6]:
sentences

[['신발', '주문', '하', '고', '싶', '어요'],
 ['주얼리', '먹', '고', '싶', '어요'],
 ['원피스', '하', '고', '싶', '어요'],
 ['주얼리', '원', '하', '어요'],
 ['바지', '어떻', '게', '하', '아야', '되', '요', '?'],
 ['바지', '어떻', '게', '하', '아야', '되', '어'],
 ['뷰티', '하', '고', '싶', '은데'],
 ['스니커즈', '알리', '어', '주', '어'],
 ['속옷', '가르치', '어', '주', '어'],
 ['생활', '하', '고', '싶', '다'],
 ['안경테', '하', '고', '싶', '은데'],
 ['신발', '하', 'ㄹ게요'],
 ['생활', '돕', '아', '주', '어'],
 ['모자', '하', 'ㄹ께', '요'],
 ['음악', '도와주', '시', '어요'],
 ['스커트', '하', 'ㄹ', '수', '있', '나요', '?'],
 ['아우터', '가능', '한', '가요'],
 ['스니커즈', '문의', '드리', '어요'],
 ['생활', '당일', '가능', '하', 'ㄴ', '가요'],
 ['취미', '하', '아', '주', '시', '어요'],
 ['예술', '하', '려구요'],
 ['티켓', '되', '었', '나요'],
 ['바지', '되', '나요'],
 ['양말', '하', 'ㄹ', '수', '있', '을까요', '?'],
 ['뷰티', '싶', '어요'],
 ['주얼리', '언제', '가능'],
 ['속옷', '언제', '가능', '하', 'ㄹ까요'],
 ['바지', '하', 'ㄴ가요', '?'],
 ['주얼리', '어떻', '게', '하', '나요'],
 ['책', '어떻', '게', '되', '나요'],
 ['아우터', '어떻', '게', '하', '나요'],
 ['아우터', '하', '는', '법'],
 ['생활', '하', 'ㄹ래'],
 ['안경테', '하', '아',

In [7]:
tags

[['B_CATEGORY', 'O', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O'],
 ['B_CATEGORY', 'O', 'O', 'O', 'O'],
 ['B_CATEGORY', 'O', 

In [8]:
print("샘플 크기 : \n", len(sentences))
print("0번 째 샘플 단어 시퀀스 : \n", sentences[0])
print("0번 째 샘플 bio 태그 : \n", tags[0])
print("샘플 단어 시퀀스 최대 길이 :", max(len(l) for l in sentences))
print("샘플 단어 시퀀스 평균 길이 :", (sum(map(len, sentences))/len(sentences)))

샘플 크기 : 
 65719
0번 째 샘플 단어 시퀀스 : 
 ['신발', '주문', '하', '고', '싶', '어요']
0번 째 샘플 bio 태그 : 
 ['B_CATEGORY', 'O', 'O', 'O', 'O', 'O']
샘플 단어 시퀀스 최대 길이 : 168
샘플 단어 시퀀스 평균 길이 : 8.646251464568847


In [9]:
# ③ 토크나이저 정의
# 단어 시퀀스의 경우 Preprocess 객체에서 생성하기 때문에 예제에서는 BIO 태그용 Tokenizer 객체 생성
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그 정보는 lower=False 소문자로 변환하지 않는다.
tag_tokenizer.fit_on_texts(tags)  # 나중에 texts_to_sequences() 등을 수행하기 전에 꼭


In [10]:
# 우선 확인
p.word_index

{'OOV': 1,
 '하': 2,
 '이': 3,
 '는': 4,
 '네': 5,
 '은': 6,
 '아': 7,
 '어요': 8,
 '요': 9,
 '반품': 10,
 '시': 11,
 '원': 12,
 '되': 13,
 '사이즈': 14,
 '있': 15,
 '가': 16,
 '게': 17,
 '얼마': 18,
 '에': 19,
 '나요': 20,
 '주문': 21,
 '만': 22,
 'ㅂ니다': 23,
 'ㄴ': 24,
 '주': 25,
 '았': 26,
 '고': 27,
 '색상': 28,
 '을': 29,
 '예': 30,
 '드리': 31,
 '는데': 32,
 '거': 33,
 '이것': 34,
 '좀': 35,
 '에요': 36,
 'ㄴ가요': 37,
 '가격': 38,
 '아요': 39,
 '죠': 40,
 '면': 41,
 '안': 42,
 '겠': 43,
 '것': 44,
 '으로': 45,
 'ㄹ': 46,
 '그': 47,
 '로': 48,
 '어떻': 49,
 '아서': 50,
 '그럼': 51,
 '어': 52,
 '도': 53,
 '상품': 54,
 '습니다': 55,
 '는데요': 56,
 '지금': 57,
 '없': 58,
 '취소': 59,
 '할인': 60,
 '맞': 61,
 '오': 62,
 '제가': 63,
 '를': 64,
 '아니': 65,
 '옷': 66,
 '나오': 67,
 '그러면': 68,
 '었': 69,
 '받': 70,
 '려고요': 71,
 '부탁': 72,
 '뭐': 73,
 'ㄹ게요': 74,
 '어떤': 75,
 '같': 76,
 '그것': 77,
 '크': 78,
 '보': 79,
 '배': 80,
 '제품': 81,
 '확인': 82,
 '수': 83,
 '님': 84,
 '가방': 85,
 '한': 86,
 'ㄹ까요': 87,
 'ㄹ려고': 88,
 '고객': 89,
 '사': 90,
 '아야': 91,
 '그냥': 92,
 '네요': 93,
 'ㄴ데': 94,
 '려고': 95,
 '

In [11]:
tag_tokenizer.word_index

{'O': 1, 'B_CATEGORY': 2, 'I': 3, 'NNP': 4}

In [12]:
# 단어사전 및 태그 사전 크기
vocab_size = len(p.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print("BIO 태그 사전 크기 :", tag_size)
print("단어 사전 크기 :", vocab_size)

BIO 태그 사전 크기 : 5
단어 사전 크기 : 2927


In [13]:
# 학습용 단어 시퀀스 생성
x_train = [p.get_wordidx_sequence(sent) for sent in sentences]
y_train = tag_tokenizer.texts_to_sequences(tags)

index_to_ner = tag_tokenizer.index_word
index_to_ner[0] = 'PAD'

In [14]:
len(x_train)

65719

In [15]:
x_train[:10]

[[98, 21, 2, 27, 124, 8],
 [912, 380, 27, 124, 8],
 [498, 2, 27, 124, 8],
 [912, 12, 2, 8],
 [142, 49, 17, 2, 91, 13, 9, 1],
 [142, 49, 17, 2, 91, 13, 52],
 [1, 2, 27, 124, 223],
 [726, 651, 52, 25, 52],
 [1206, 1, 52, 25, 52],
 [2268, 2, 27, 124, 107]]

In [16]:
y_train[:10]

[[2, 1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 1, 1, 1],
 [2, 1, 1, 1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 1, 1, 1, 1]]

In [17]:
index_to_ner   # 0: 'PAD'  가 추가되었다

{1: 'O', 2: 'B_CATEGORY', 3: 'I', 4: 'NNP', 0: 'PAD'}

In [18]:
# ⑤ 시퀀스 패딩 처리
max_len = 40
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)


In [19]:
x_train,y_train

(array([[ 98,  21,   2, ...,   0,   0,   0],
        [912, 380,  27, ...,   0,   0,   0],
        [498,   2,  27, ...,   0,   0,   0],
        ...,
        [142, 532, 685, ...,   0,   0,   0],
        [142, 532, 685, ...,   0,   0,   0],
        [142, 532, 685, ...,   0,   0,   0]]),
 array([[2, 1, 1, ..., 0, 0, 0],
        [2, 1, 1, ..., 0, 0, 0],
        [2, 1, 1, ..., 0, 0, 0],
        ...,
        [2, 1, 1, ..., 0, 0, 0],
        [2, 1, 1, ..., 0, 0, 0],
        [2, 1, 1, ..., 0, 0, 0]]))

In [20]:
x_train.shape, y_train.shape

((65719, 40), (65719, 40))

In [21]:
# # ⑥ 학습데이터와 테스트 데이터를 7:3의 비율로 분리 ( 한 번 만 실 행)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train,
                                                    test_size=.3,
                                                    random_state=1234)


In [22]:
len(x_train), len(x_test), len(y_train), len(y_test)

(46003, 19716, 46003, 19716)

In [23]:
y_train

array([[2, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [2, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [24]:
tag_size

5

In [25]:
# 출력 데이터를 one-hot encoding
# # ★ 한번만 실행할것!
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)  # tag_size. BIO 태그 사전 크기, 
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)    # 현재 tag_size 는 10 이다

In [26]:
y_train.shape , y_test.shape 
# y array[3,1,1,...,0,0,0] ... 에서 3 을 one-hot encoding
# 1 을 one-hot encoding  .....
# 하나하나 한것

((46003, 40, 5), (19716, 40, 5))

In [27]:
print("학습 샘플 시퀀스 형상 : ", x_train.shape)
print("학습 샘플 레이블 형상 : ", y_train.shape)
print("테스트 샘플 시퀀스 형상 : ", x_test.shape)
print("테스트 샘플 레이블 형상 : ", y_test.shape)

학습 샘플 시퀀스 형상 :  (46003, 40)
학습 샘플 레이블 형상 :  (46003, 40, 5)
테스트 샘플 시퀀스 형상 :  (19716, 40)
테스트 샘플 레이블 형상 :  (19716, 40, 5)


In [28]:
# ⑦ 모델 정의 (Bi-LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])




In [29]:
model_file = os.path.join(r'./models/ner', 'ner_model.h5')
model_file  

'./models/ner\\ner_model.h5'

In [88]:
# 학습하기
model.fit(x_train,y_train, batch_size = 128, epochs=10)

Epoch 1/10
360/360 [==============================] - 117s 305ms/step - loss: 0.0174 - accuracy: 0.9756
Epoch 2/10
360/360 [==============================] - 120s 333ms/step - loss: 0.0048 - accuracy: 0.9929
Epoch 3/10
360/360 [==============================] - 121s 335ms/step - loss: 0.0042 - accuracy: 0.9935
Epoch 4/10
360/360 [==============================] - 122s 339ms/step - loss: 0.0039 - accuracy: 0.9939
Epoch 5/10
360/360 [==============================] - 125s 347ms/step - loss: 0.0037 - accuracy: 0.9941
Epoch 6/10
360/360 [==============================] - 179s 499ms/step - loss: 0.0036 - accuracy: 0.9942
Epoch 7/10
360/360 [==============================] - 228s 633ms/step - loss: 0.0036 - accuracy: 0.9942
Epoch 8/10
360/360 [==============================] - 20267s 56s/step - loss: 0.0034 - accuracy: 0.9945
Epoch 9/10
360/360 [==============================] - 230s 639ms/step - loss: 0.0032 - accuracy: 0.9946
Epoch 10/10
360/360 [==============================] - 231s 641m

In [106]:
print("평가 결과 : ", model.evaluate(x_test, y_test)[1])
model.save(os.path.join('./models/ner', 'ner_model.h5'))  # 학습 완료된 모델 저장

617/617 [==============================] - 34s 54ms/step - loss: 0.0050 - accuracy: 0.9930
평가 결과 :  0.9930064678192139


In [30]:
# 의도 분류 모델 불러오기
import os
model_name=os.path.join('./models/ner', 'ner_model.h5')
model = load_model(model_name)

In [31]:
# 시퀀스를 NER 태그로 변환
def sequences_to_tag(sequences):  # 예측값을 index_to_ner를 사용하여 태깅 정보로 변경하는 함수.
    result = []
    for sequence in sequences:  # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
        temp = []
        for pred in sequence:  # 시퀀스로부터 예측값을 하나씩 꺼낸다.
            pred_index = np.argmax(pred)  # 예를 들어 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))  # 'PAD'는 'O'로 변경
        result.append(temp)
    return result

In [32]:
from seqeval.metrics import f1_score, classification_report

In [93]:
# 테스트 데이터셋의 NER 예측
# F1 score 를 게산하기 위해 모델의 predict() 함수를 통해 테스트용 데이터 셋의 결과를 예측합니다.
# 해당 함수의 입력은 시퀀스 번호로 인코딩 된 테스트 단어 시퀀스 (ndarray 배열)을 사용
# 해당 함수의 결과는 예측된 NER 태그 정보가 담긴 ndarray 배열을 리턴
# y_predicted = model.predict(x_test)

In [94]:
# pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
# test_tags = sequences_to_tag(y_test)    # 실제 NER

In [95]:
# F1 평가 결과
# seqeval.metrics 모듈의 classification+_report 함수를 통해 NER 태그 별로 계산된 precision, recall 그리고 f1_score 값을 출력

# print(classification_report(test_tags, pred_tags))

# # f1_score() 함수 만으로 f1 score 값만 출력 가능
# print("F1-score: {:.1%}".format(f1_score(test_tags, pred_tags)))

In [33]:
# 질의어 개체명 인식할.. 질의어
query = '레그웨어'
pos = p.pos(query)
pos

[('레그웨어', 'NNG')]

In [34]:
keywords = p.get_keywords(pos, without_tag=True)
keywords

['레그웨어']

In [35]:
sequences = [p.get_wordidx_sequence(keywords)]
sequences

[[1]]

In [36]:
# 패딩처리
max_len = 40
padded_seqs = preprocessing.sequence.pad_sequences(sequences, padding="post", value=0, maxlen=max_len)
padded_seqs

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [37]:
# 문장하나
# 첫번째 batch
padded_seqs[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [38]:
# 예측하기
predict = model.predict(padded_seqs)
predict

array([[[4.9103404e-05, 1.3554308e-01, 8.6418992e-01, 1.9592739e-04,
         2.1986092e-05],
        [1.9074255e-01, 2.6024261e-01, 1.8529096e-01, 1.7231458e-01,
         1.9140930e-01],
        [1.9074255e-01, 2.6024261e-01, 1.8529096e-01, 1.7231458e-01,
         1.9140930e-01],
        [1.9074255e-01, 2.6024261e-01, 1.8529096e-01, 1.7231458e-01,
         1.9140930e-01],
        [1.9074255e-01, 2.6024261e-01, 1.8529096e-01, 1.7231458e-01,
         1.9140930e-01],
        [1.9074255e-01, 2.6024261e-01, 1.8529096e-01, 1.7231458e-01,
         1.9140930e-01],
        [1.9074255e-01, 2.6024261e-01, 1.8529096e-01, 1.7231458e-01,
         1.9140930e-01],
        [1.9074255e-01, 2.6024261e-01, 1.8529096e-01, 1.7231458e-01,
         1.9140930e-01],
        [1.9074255e-01, 2.6024261e-01, 1.8529096e-01, 1.7231458e-01,
         1.9140930e-01],
        [1.9074255e-01, 2.6024261e-01, 1.8529096e-01, 1.7231458e-01,
         1.9140930e-01],
        [1.9074255e-01, 2.6024261e-01, 1.8529096e-01, 1.7231

In [39]:
predict_class = tf.math.argmax(predict, axis=-1)
predict_class

<tf.Tensor: shape=(1, 40), dtype=int64, numpy=
array([[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
      dtype=int64)>

In [40]:
tags = [index_to_ner[i] for i in predict_class.numpy()[0]]
print(tags)

['B_CATEGORY', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [41]:
# 개체명 클래스 예측 결과
predicts = list(zip(keywords, tags))
predicts

[('레그웨어', 'B_CATEGORY')]

In [42]:
# 예측한 tag 들
tags = []
for tag_idx in predict_class.numpy()[0]:
    if tag_idx == 1: continue   # O 태그 제외
    tags.append(index_to_ner[tag_idx])

tags

['B_CATEGORY']